In [1]:
from utils.utils import *
import matplotlib.pyplot as plt
import utils.language_helpers as lh
from utils.bio_utils import *
import pandas as pd
import numpy as np
import random
import numpy as np

In [2]:
def load_gensamples(path):
    with open(path,encoding='utf-8') as f:
        content = f.read()
        lines = content.split('\n')
        return lines

In [3]:
#real sample
real_datas,_,__ = lh.load_dataset(max_length=156,max_n_examples=8192,data_dir='./data/AMP_dataset.fa')
real_lines = []
for data in real_datas:
        line = ''
        for char in data:
                if char.isalpha():
                        line += char
        real_lines.append(line)
print(len(real_lines))

real_p = geneToProtein(real_lines,verbose=False)

print(len(real_p))


loading dataset...
('A', 'T', 'G', 'C', 'A', 'G', 'C', 'T', 'G', 'C', 'C', 'T', 'A', 'T', 'A', 'T', 'G', 'C', 'G', 'G', 'T', 'G', 'A', 'G', 'A', 'C', 'G', 'T', 'G', 'T', 'G', 'T', 'C', 'C', 'T', 'T', 'G', 'G', 'G', 'G', 'G', 'A', 'T', 'G', 'C', 'T', 'A', 'C', 'A', 'C', 'G', 'C', 'C', 'G', 'A', 'A', 'T', 'T', 'G', 'C', 'C', 'G', 'T', 'T', 'G', 'C', 'C', 'A', 'A', 'T', 'A', 'C', 'C', 'C', 'C', 'A', 'T', 'T', 'T', 'G', 'C', 'G', 'T', 'C', 'C', 'G', 'T', 'T', 'A', 'G', '\t', '1', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P')
loaded 5200 lines in dataset
5200
4930


In [4]:
#before train
# pre_lines = []
# for i in range(2000):
#     j = i+10000
#     if j % 100 == 99:
#         path = './samples/realProt_50aa/sampled_{}.txt'.format(j)
#         temp = load_gensamples(path)
#         for line in temp:
#             pre_lines.append(line)

path = './LargeSample/pre_gen.txt'
pre_lines = load_gensamples(path)
print(len(pre_lines))

pre_p = geneToProtein(pre_lines,verbose=False)

print(len(pre_p))

8193
6476


In [5]:
#after train
# fb_lines = []
# for i in range(10):
#     path = './samples/fbgan_amp_demo/sampled_{}_preds.txt'.format(i+90)
#     temp = load_gensamples(path)
#     for line in temp:
#         fb_lines.append(line)

# fb_lines = load_gensamples('samples/fbgan_mode/sampled_120_preds.txt')
# fb_lines = load_gensamples('samples/fbgan_amp_demo/sampled_98_preds.txt.txt')

path = './LargeSample/fb_gen.txt'
fb_lines = load_gensamples(path)
print(len(fb_lines))

fb_p = geneToProtein(fb_lines,verbose=False)

print(len(fb_p))

8193
6473


In [6]:
def calcNLD(str1,str2):

    matrix = [[ i + j for j in range(len(str2) + 1)] for i in range(len(str1) + 1)]
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            if(str1[i-1] == str2[j-1]):
                d = 0
            else:
                d = 1
            matrix[i][j] = min(matrix[i-1][j]+1, matrix[i][j-1]+1, matrix[i-1][j-1]+d)
    return float(matrix[len(str1)][len(str2)])/max(len(str1),len(str2))

def calcNLDs(aim1,aim2):
    LSDs = []
    for line1 in aim1:
        lsd = 0.0
        cnt = 0.0
        for line2 in aim2:
            if line2 != '':
                lsd += calcNLD(line1[0],line2[0])
                cnt+=1
        LSDs.append(lsd/cnt)
        
    return LSDs


In [7]:
# from random import shuffle
# from numpy import round,mean
# shuffle(real_p)
# shuffle(pre_p)
# shuffle(fb_p)
# real_p = real_p[0:500]
# pre_p = pre_p[0:500]
# fb_p = fb_p[0:500]

# pre2real = calcNLDs(real_p,pre_p)

# fb2real = calcNLDs(real_p,fb_p)

# pre2real = round(pre2real)

# fb2real = round(fb2real)


# print(mean(pre2real))
# print(mean(fb2real))


In [8]:
# import pandas as pd 
# print(len(pre2real))
# print(len(fb2real))
# df = pd.DataFrame(
#     pre2real,
#     columns=['Pre-train']
# )
# df['Feedback train'] = fb2real

# bin = 20
# ax = df.plot.hist(figsize=(10,5),bins=bin, alpha=0.5)
# fig = ax.get_figure()
# fig.savefig('./AMP/EditDis.png')

In [9]:
pre_df = pd.DataFrame(pre_p)
fb_df = pd.DataFrame(fb_p)
real_df = pd.DataFrame(real_p)
rs = np.random.RandomState(12345)

for i in range(5):
    for j in range(3):
        idxi = i+1
        idxj = j+1
        nowidx = idxi*10+idxj
        seed = rs.randint(0,65536)
        pre_sample = pre_df.sample(500,random_state=seed).values.tolist()
        seed = rs.randint(0,65536)
        fb_sample = fb_df.sample(500,random_state=seed).values.tolist()
        seed = rs.randint(0,65536)
        real_sample = real_df.sample(500,random_state=seed).values.tolist()
        pre2real = calcNLDs(real_sample,pre_sample)
        fb2real = calcNLDs(real_sample,fb_sample)
        print(np.mean(pre2real))
        print(np.mean(fb2real))
        df = pd.DataFrame(
                pre2real,
                columns=['Pre-train']
        )
        df['Feedback train'] = fb2real
        bin = 20
        ax = df.plot.hist(figsize=(10,5),bins=bin, alpha=0.5)
        fig = ax.get_figure()
        fig.savefig('./AMP/EditDis{}.png'.format(nowidx))

0.8719352632595906
0.8539201232697796
0.8697558854187258
0.8549816080878627
0.8687077200510775
0.8552186670726928
0.8687720059470517
0.8552844348806227
0.8731515077731768
0.8530544958332956
0.8717635977214409
0.8529169198310875
0.8707618525303574
0.854627270901703
0.8692558977653868
0.8507849165336981
0.871839017874562
0.8555927433831377
0.8716014472275302
0.8526813486117386
0.8690815611342615
0.8524404630229706
0.8691482348084457
0.8536214942093591
0.8709798306161202
0.8532445104132064
0.8709808440244219
0.8547420522790226
0.8669876937184433
0.8556936059962797
